# Project Template - adapted

## Imports

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

Be sure to start the stream on Kafka!

## Create Spark Session

In [2]:
spark = (
    SparkSession.builder
        .appName("Project DPA")
        .master("local[*]")   
        .config("spark.jars.packages",
                "org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.0")
        .config("spark.sql.repl.eagerEval.enabled", True)
        .getOrCreate()
)

spark.sparkContext.setLogLevel("WARN")

Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
:: loading settings :: url = jar:file:/usr/local/lib/python3.9/dist-packages/pyspark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-e28531f5-1045-4bd5-ab4a-cf895cbc0fb9;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.0.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.0.0 in central
	found org.apache.kafka#kafka-clients;2.4.1 in central
	found com.github.luben#zstd-jni;1.4.4-3 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.7.5 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.commons#commons-pool2;2.6.2 in central
:: resolution report :: resolve 258ms :: artifacts dl 3m

## Schema Definiton

In [3]:
schema = StructType([
    StructField("name", StringType()),
    StructField("price", FloatType()),
    StructField("n_shares", IntegerType()),
    StructField("timestamp", StringType())
])

# Kafka Stream Ingestion

In [4]:
# Read from Kafka topic "stock"
raw_df = (
    spark.readStream
         .format("kafka")
         .option("kafka.bootstrap.servers", "kafka1:9092")
         .option("subscribe", "stock")
         .option("startingOffsets", "latest")
         .load()
)

# Kafka sends bytes → convert to string
json_df = raw_df.selectExpr("CAST(value AS STRING)")

# Parse JSON using the schema
parsed_df = json_df.select(
    from_json(col("value"), schema).alias("data")
).select("data.*")

# Convert timestamp string into actual timestamp type
parsed_df = parsed_df.withColumn(
    "timestamp", to_timestamp("timestamp", "yyyy-MM-dd'T'HH:mm:ss'Z'")
)

# Make the dataset available to all following tasks
parsed_df.createOrReplaceTempView("stocks_stream")

# Start Streaming Output

This is optional but just to make sure that Spark is actually receiving data

In [ ]:
query = (
    parsed_df.writeStream
             .format("console")
             .outputMode("append")
             .option("truncate", False)
             .start()
)

## The assignment starts here

Remark: We consider a day being 5 minutes so we do not have to wait an actual day for the code results. Hence, an hour corresponds to 12.5 seconds.

### Task 1

In [5]:
task1 = (
    parsed_df
        .withWatermark("timestamp", "10 minutes")
        .groupBy(window("timestamp", "5 minutes"), "name")
        .agg(variance("price").alias("variance_daily_price"),
                avg("price").alias("average_daily_price"))
        .orderBy("variance_daily_price")
)

task1_query = (
    task1.writeStream
         .format("console")
         .outputMode("complete")
         .option("truncate", False)
         .start()
)

25/11/24 16:17:39 WARN StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-aba932fe-b146-47dc-9c4b-b426e2e1d179. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.


-------------------------------------------
Batch: 0
-------------------------------------------
+------------------------------------------+----+--------------------+-------------------+
|window                                    |name|variance_daily_price|average_daily_price|
+------------------------------------------+----+--------------------+-------------------+
|[2025-11-24 16:15:00, 2025-11-24 16:20:00]|AMT |NaN                 |97.31999969482422  |
|[2025-11-24 16:15:00, 2025-11-24 16:20:00]|PRGO|NaN                 |156.4600067138672  |
+------------------------------------------+----+--------------------+-------------------+



-------------------------------------------
Batch: 1
-------------------------------------------
+------------------------------------------+----+---------------------+-------------------+
|window                                    |name|variance_daily_price |average_daily_price|
+------------------------------------------+----+---------------------+-------------------+
|[2025-11-24 16:15:00, 2025-11-24 16:20:00]|F   |0.0018000823984039016|16.170000076293945 |
|[2025-11-24 16:15:00, 2025-11-24 16:20:00]|ABT |0.009799914550967515 |47.63999938964844  |
|[2025-11-24 16:15:00, 2025-11-24 16:20:00]|PNR |0.009799914550967515 |67.55000305175781  |
|[2025-11-24 16:15:00, 2025-11-24 16:20:00]|DE  |0.012800585944205523 |89.52999877929688  |
|[2025-11-24 16:15:00, 2025-11-24 16:20:00]|JNJ |0.019999389653094113 |103.0              |
|[2025-11-24 16:15:00, 2025-11-24 16:20:00]|GRMN|0.026449894714460243 |49.704999923706055 |
|[2025-11-24 16:15:00, 2025-11-24 16:20:00]|EXR |0.03920179445412941  |65.7

-------------------------------------------
Batch: 2
-------------------------------------------
+------------------------------------------+----+---------------------+-------------------+
|window                                    |name|variance_daily_price |average_daily_price|
+------------------------------------------+----+---------------------+-------------------+
|[2025-11-24 16:15:00, 2025-11-24 16:20:00]|SYF |0.0                  |31.549999237060547 |
|[2025-11-24 16:15:00, 2025-11-24 16:20:00]|PDCO|2.000091553782113E-4 |47.6299991607666   |
|[2025-11-24 16:15:00, 2025-11-24 16:20:00]|F   |0.0018000823984039016|16.170000076293945 |
|[2025-11-24 16:15:00, 2025-11-24 16:20:00]|TWX |0.004801958129974082 |84.46100234985352  |
|[2025-11-24 16:15:00, 2025-11-24 16:20:00]|RSG |0.004999847413273528 |40.619998931884766 |
|[2025-11-24 16:15:00, 2025-11-24 16:20:00]|ABT |0.009799914550967515 |47.63999938964844  |
|[2025-11-24 16:15:00, 2025-11-24 16:20:00]|DE  |0.012800585944205523 |89.5

-------------------------------------------
Batch: 3
-------------------------------------------
+------------------------------------------+----+---------------------+-------------------+
|window                                    |name|variance_daily_price |average_daily_price|
+------------------------------------------+----+---------------------+-------------------+
|[2025-11-24 16:15:00, 2025-11-24 16:20:00]|SYF |0.0                  |31.549999237060547 |
|[2025-11-24 16:15:00, 2025-11-24 16:20:00]|PDCO|2.000091553782113E-4 |47.6299991607666   |
|[2025-11-24 16:15:00, 2025-11-24 16:20:00]|F   |0.0018000823984039016|16.170000076293945 |
|[2025-11-24 16:15:00, 2025-11-24 16:20:00]|TWX |0.004801958129974082 |84.46100234985352  |
|[2025-11-24 16:15:00, 2025-11-24 16:20:00]|RSG |0.004999847413273528 |40.619998931884766 |
|[2025-11-24 16:15:00, 2025-11-24 16:20:00]|ABT |0.009799914550967515 |47.63999938964844  |
|[2025-11-24 16:15:00, 2025-11-24 16:20:00]|JNJ |0.019999389653094113 |103.

...

-------------------------------------------
Batch: 136
-------------------------------------------
+------------------------------------------+----+--------------------+-------------------+
|window                                    |name|variance_daily_price|average_daily_price|
+------------------------------------------+----+--------------------+-------------------+
|[2025-11-24 16:15:00, 2025-11-24 16:20:00]|DWDP|0.20479960937518626 |71.37000274658203  |
|[2025-11-24 16:15:00, 2025-11-24 16:20:00]|AES |0.7456805482096444  |11.901153931250938 |
|[2025-11-24 16:20:00, 2025-11-24 16:25:00]|AES |1.3239604733775268  |11.617110013961792 |
|[2025-11-24 16:20:00, 2025-11-24 16:25:00]|KEY |1.3490028263934297  |12.868235139285817 |
|[2025-11-24 16:20:00, 2025-11-24 16:25:00]|BHGE|1.6688689036078965  |35.589999198913574 |
|[2025-11-24 16:15:00, 2025-11-24 16:20:00]|NAVI|1.9299861023425338  |14.489408175150553 |
|[2025-11-24 16:20:00, 2025-11-24 16:25:00]|DWDP|2.183425416063983   |72.293334960

-------------------------------------------
Batch: 137
-------------------------------------------
+------------------------------------------+----+--------------------+-------------------+
|window                                    |name|variance_daily_price|average_daily_price|
+------------------------------------------+----+--------------------+-------------------+
|[2025-11-24 16:15:00, 2025-11-24 16:20:00]|DWDP|0.20479960937518626 |71.37000274658203  |
|[2025-11-24 16:15:00, 2025-11-24 16:20:00]|AES |0.7456805482096444  |11.901153931250938 |
|[2025-11-24 16:20:00, 2025-11-24 16:25:00]|AES |1.3239604733775268  |11.617110013961792 |
|[2025-11-24 16:20:00, 2025-11-24 16:25:00]|BHGE|1.6688689036078965  |35.589999198913574 |
|[2025-11-24 16:15:00, 2025-11-24 16:20:00]|NAVI|1.9299861023425338  |14.489408175150553 |
|[2025-11-24 16:20:00, 2025-11-24 16:25:00]|DWDP|2.183425416063983   |72.2933349609375   |
|[2025-11-24 16:15:00, 2025-11-24 16:20:00]|BHF |2.184050318908703   |58.765001296

-------------------------------------------
Batch: 138
-------------------------------------------
+------------------------------------------+----+--------------------+-------------------+
|window                                    |name|variance_daily_price|average_daily_price|
+------------------------------------------+----+--------------------+-------------------+
|[2025-11-24 16:15:00, 2025-11-24 16:20:00]|DWDP|0.20479960937518626 |71.37000274658203  |
|[2025-11-24 16:15:00, 2025-11-24 16:20:00]|AES |0.7456805482096444  |11.901153931250938 |
|[2025-11-24 16:20:00, 2025-11-24 16:25:00]|AES |1.259429594619221   |11.608200027829124 |
|[2025-11-24 16:20:00, 2025-11-24 16:25:00]|BHGE|1.6688689036078965  |35.589999198913574 |
|[2025-11-24 16:15:00, 2025-11-24 16:20:00]|NAVI|1.9299861023425338  |14.489408175150553 |
|[2025-11-24 16:20:00, 2025-11-24 16:25:00]|DWDP|2.183425416063983   |72.2933349609375   |
|[2025-11-24 16:15:00, 2025-11-24 16:20:00]|BHF |2.184050318908703   |58.765001296

-------------------------------------------
Batch: 139
-------------------------------------------
+------------------------------------------+----+--------------------+-------------------+
|window                                    |name|variance_daily_price|average_daily_price|
+------------------------------------------+----+--------------------+-------------------+
|[2025-11-24 16:15:00, 2025-11-24 16:20:00]|DWDP|0.20479960937518626 |71.37000274658203  |
|[2025-11-24 16:15:00, 2025-11-24 16:20:00]|AES |0.7456805482096444  |11.901153931250938 |
|[2025-11-24 16:20:00, 2025-11-24 16:25:00]|AES |1.259429594619221   |11.608200027829124 |
|[2025-11-24 16:20:00, 2025-11-24 16:25:00]|BHGE|1.6688689036078965  |35.589999198913574 |
|[2025-11-24 16:15:00, 2025-11-24 16:20:00]|NAVI|1.9299861023425338  |14.489408175150553 |
|[2025-11-24 16:20:00, 2025-11-24 16:25:00]|DWDP|2.183425416063983   |72.2933349609375   |
|[2025-11-24 16:15:00, 2025-11-24 16:20:00]|BHF |2.184050318908703   |58.765001296

-------------------------------------------
Batch: 140
-------------------------------------------
+------------------------------------------+----+--------------------+-------------------+
|window                                    |name|variance_daily_price|average_daily_price|
+------------------------------------------+----+--------------------+-------------------+
|[2025-11-24 16:15:00, 2025-11-24 16:20:00]|DWDP|0.20479960937518626 |71.37000274658203  |
|[2025-11-24 16:15:00, 2025-11-24 16:20:00]|AES |0.7456805482096444  |11.901153931250938 |
|[2025-11-24 16:20:00, 2025-11-24 16:25:00]|AES |1.259429594619221   |11.608200027829124 |
|[2025-11-24 16:20:00, 2025-11-24 16:25:00]|BHGE|1.6688689036078965  |35.589999198913574 |
|[2025-11-24 16:15:00, 2025-11-24 16:20:00]|NAVI|1.9299861023425338  |14.489408175150553 |
|[2025-11-24 16:20:00, 2025-11-24 16:25:00]|DWDP|2.183425416063983   |72.2933349609375   |
|[2025-11-24 16:15:00, 2025-11-24 16:20:00]|BHF |2.184050318908703   |58.765001296